# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [31]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [32]:
weather_df = pd.read_csv("new_city_data.csv").reset_index(drop=True).drop(columns="Unnamed: 0")
weather_df

,City,Latitude,Longitude,Temperature,Max_Temp,Humidity,Cloudiness,Wind,Country
0,Morón,-34.6534,-58.6198,77.43,79.00,62,57,7.00,AR
1,Huanren,41.2647,125.3667,20.68,20.68,95,0,3.09,CN
2,Butaritari,3.0707,172.7902,81.59,81.59,76,100,16.60,KI
3,Necochea,-38.5473,-58.7368,70.00,70.00,69,22,8.01,AR
4,Ushuaia,-54.8000,-68.3000,75.20,75.20,33,0,19.57,AR
...,...,...,...,...,...,...,...,...,...
571,Victor Harbor,-35.5667,138.6167,83.28,86.00,32,25,10.80,AU
572,Ürümqi,43.8010,87.6005,15.80,15.80,92,75,2.24,CN
573,Mahon,39.8885,4.2658,48.20,48.20,87,0,4.61,ES
574,Viedma,-40.8135,-62.9967,79.00,79.00,42,18,1.01,AR


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [28]:
gmaps.configure(api_key=g_key)

In [29]:
#Use the Lat and Lng as locations and Humidity as the weight

locations = weather_df[['Latitude', 'Longitude']]
humidity = weather_df['Humidity']

#Add Heatmap layer to map
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                dissipating=False, max_intensity=10,
                                 point_radius=1)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [59]:
# Narrow down the DataFrame to find your ideal weather condition. For example:
# A max temperature lower than 80 degrees but higher than 70.
# Wind speed less than 10 mph.
# Zero cloudiness.
# Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.

new_temp = (weather_df.Max_Temp <= 80) & (weather_df.Max_Temp >= 60)
new_wind = weather_df.Wind < 15
new_cloud = weather_df.Cloudiness == 0
new_criteria = new_temp & new_wind & new_cloud


new_weather_df = weather_df[new_criteria]
new_weather_df = new_weather_df.dropna()
new_weather_df = new_weather_df.reset_index()
new_weather_df

,index,City,Latitude,Longitude,Temperature,Max_Temp,Humidity,Cloudiness,Wind,Country
0,17,Kiryat Gat,31.6100,34.7642,55.33,60.01,100,0,5.46,IL
1,23,Illapel,-31.6308,-71.1653,76.84,76.84,44,0,7.63,CL
2,67,Maţāy,28.4190,30.7792,60.30,60.30,47,0,9.33,EG
3,72,Punta Arenas,-53.1500,-70.9167,66.09,66.20,55,0,10.36,CL
4,106,Det Udom,14.9060,105.0784,62.60,62.60,82,0,6.91,TH
5,119,Myanaung,18.2833,95.3167,63.95,63.95,49,0,4.74,MM
6,152,Christchurch,-43.5333,172.6333,67.78,69.01,30,0,14.97,NZ
7,167,Saint-Philippe,-21.3585,55.7679,73.42,78.80,69,0,5.75,RE
8,184,Riyadh,24.6877,46.7219,61.72,64.40,29,0,5.75,SA
9,190,Telimele,10.9167,-13.3333,69.15,69.15,71,0,2.51,GN


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [60]:
#store variable and add hotel name to df
hotel_df = pd.DataFrame(new_weather_df)
hotel_df["Hotel Name"] = ""
hotel_df

,index,City,Latitude,Longitude,Temperature,Max_Temp,Humidity,Cloudiness,Wind,Country,Hotel Name
0,17,Kiryat Gat,31.6100,34.7642,55.33,60.01,100,0,5.46,IL,
1,23,Illapel,-31.6308,-71.1653,76.84,76.84,44,0,7.63,CL,
2,67,Maţāy,28.4190,30.7792,60.30,60.30,47,0,9.33,EG,
3,72,Punta Arenas,-53.1500,-70.9167,66.09,66.20,55,0,10.36,CL,
4,106,Det Udom,14.9060,105.0784,62.60,62.60,82,0,6.91,TH,
5,119,Myanaung,18.2833,95.3167,63.95,63.95,49,0,4.74,MM,
6,152,Christchurch,-43.5333,172.6333,67.78,69.01,30,0,14.97,NZ,
7,167,Saint-Philippe,-21.3585,55.7679,73.42,78.80,69,0,5.75,RE,
8,184,Riyadh,24.6877,46.7219,61.72,64.40,29,0,5.75,SA,
9,190,Telimele,10.9167,-13.3333,69.15,69.15,71,0,2.51,GN,


In [72]:
#set parameters
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "radius": 5000,
    "type": "lodging",
    "rankby": "distance",
    "key": g_key
}


#hit google places api for each cities coordinates
for index, row in hotel_df.iterrows():
    lat = row["Latitude"]
    lng = row["Longitude"]
    city_name = row["City"]
    
    params["location"] = f"{lat},{lng}"
    
    print(f"Retrieving results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    results = response['results']
    
#store hotel results in data frame
    try:
        print(f"The closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']
    except (KeyError, IndexError):
        print("No results")
    print("------------")

Retrieving results for Index 0: Kiryat Gat.
No results
------------
Retrieving results for Index 1: Illapel.
No results
------------
Retrieving results for Index 2: Maţāy.
No results
------------
Retrieving results for Index 3: Punta Arenas.
No results
------------
Retrieving results for Index 4: Det Udom.
No results
------------
Retrieving results for Index 5: Myanaung.
No results
------------
Retrieving results for Index 6: Christchurch.
No results
------------
Retrieving results for Index 7: Saint-Philippe.
No results
------------
Retrieving results for Index 8: Riyadh.
No results
------------
Retrieving results for Index 9: Telimele.
No results
------------
Retrieving results for Index 10: Ubari.
No results
------------
Retrieving results for Index 11: Saldanha.
No results
------------
Retrieving results for Index 12: Salalah.
No results
------------
Retrieving results for Index 13: Tūkrah.
No results
------------
Retrieving results for Index 14: Jaisalmer.
No results
------------


In [73]:
hotel_df

,index,City,Latitude,Longitude,Temperature,Max_Temp,Humidity,Cloudiness,Wind,Country,Hotel Name
0,17,Kiryat Gat,31.6100,34.7642,55.33,60.01,100,0,5.46,IL,
1,23,Illapel,-31.6308,-71.1653,76.84,76.84,44,0,7.63,CL,
2,67,Maţāy,28.4190,30.7792,60.30,60.30,47,0,9.33,EG,
3,72,Punta Arenas,-53.1500,-70.9167,66.09,66.20,55,0,10.36,CL,
4,106,Det Udom,14.9060,105.0784,62.60,62.60,82,0,6.91,TH,
5,119,Myanaung,18.2833,95.3167,63.95,63.95,49,0,4.74,MM,
6,152,Christchurch,-43.5333,172.6333,67.78,69.01,30,0,14.97,NZ,
7,167,Saint-Philippe,-21.3585,55.7679,73.42,78.80,69,0,5.75,RE,
8,184,Riyadh,24.6877,46.7219,61.72,64.40,29,0,5.75,SA,
9,190,Telimele,10.9167,-13.3333,69.15,69.15,71,0,2.51,GN,


In [43]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [45]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))